In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import tiktoken
import os, re, json, glob, torch
import numpy as np
from tqdm.auto import tqdm
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, pipeline,
    AutoModelForSequenceClassification
)
from sentence_transformers import SentenceTransformer, util
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from PyPDF2 import PdfReader
import gc

GUIDELINE_FILE = "/home/alpaco/autosry/rnd_guideline.json"
RAG_JSON_FILES = ["/home/alpaco/autosry/rag_chunks500_50.json"]
LAW_DIR = "/home/alpaco/autosry/reference_file"

E5_NAME = "intfloat/multilingual-e5-large"
GEN_NAME = "skt/A.X-4.0-Light"
NLI_MODEL = "MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli"

DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"

# GPU 메모리 감지 → 안전 토큰 설정
GEN_MAX_NEW_TOKENS = 3000
if torch.cuda.is_available():
    free, total = torch.cuda.mem_get_info()
    if total < 12 * 1024**3:  # 12GB 미만 GPU는 토큰 수 축소
        GEN_MAX_NEW_TOKENS = 1500
torch.cuda.empty_cache()

GEN_DO_SAMPLE = False

def first_n_lines(text: str, n_chars=400):
    t = " ".join(str(text).split())
    return t[:n_chars]

def clean_generated_text(text: str) -> str:
    text = re.sub(r'[•●▪▶◇◆□▪️▫️–]', ' ', text)
    text = re.sub(r'^\s*[-#*]+\s*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\n{3,}', '\n\n', text)
    text = re.sub(r'\s+([\.,;:])', r'\1', text)
    return text.strip()

def has_number(text: str):
    return bool(re.search(r"\d", text))

sections = [
    {"section": "연구기획과제의 개요",
        "role": "제안서 총괄 에디터",
        "query": "과제의 목적, 필요성, 기대효과를 논리적으로 연결하여 핵심 내용을 500자 내외로 명료하게 요약함. 평가자가 과제의 전체 구조를 한눈에 이해할 수 있도록 기술함.",
        "constraints": ["500자 내외", "논리적 연결 구조 유지", "핵심 요약 중심"],
        "output_format": "서술문"},
    {"section": "연구개발과제의 배경",
        "role": "R&D 기획 전문가",
        "query": "제공된 선행연구, 시장 동향, 정책자료를 근거로 본 과제가 추진되어야 하는 당위성과 RFP(공고문) 부합성을 논리적으로 제시함.",
        "constraints": ["데이터 근거 포함", "RFP 문항 부합성 명시"],
        "output_format": "서술문"},
    {"section": "연구개발과제의 필요성",
        "role": "산업분석가",
        "query": "데이터와 사례를 근거로 현재 기술적·산업적 문제점을 제시하고, 본 과제가 이를 해결해야 하는 필요성을 인과적으로 서술함.",
        "constraints": ["인과관계 구조", "데이터 근거 제시"],
        "output_format": "서술문"},
    {"section": "기술개발 핵심어(키워드)",
        "role": "기술 네이밍 전략가",
        "query": "과제의 정체성과 핵심 기술을 대표하는 키워드 5개를 국문·영문 공식 명칭으로 제시함. 각 용어는 국제 표준 또는 학술 정의를 근거로 하며, 선정 사유를 간단히 명시함.",
        "constraints": ["국문·영문 병기", "국제 표준 기반 정의 포함", "5개 이내"],
        "output_format": "표(키워드/영문명/정의/출처)"},
    {"section": "연구개발 목표",
        "role": "R&D PMO",
        "query": "과제의 최종 목표를 500자 내외로 명확히 기술함. 핵심 성능지표(KPI), 달성 기준(수치·단위·마일스톤), 검증 방법을 포함하며, 모호한 표현은 사용하지 않음.",
        "constraints": ["정량화된 수치 포함", "KPI 및 검증방법 명시"],
        "output_format": "서술문 + 표(KPI/단위/기준/검증방법)"},
        {"section": "연구개발 내용",
            "role": "기술총괄자(Tech Lead)",
            "query": "전체 연구 범위를 1,000자 내외로 체계적으로 기술함. 핵심 기술요소, 세부 과제 구조, 데이터 및 시스템 흐름, 성능 평가 계획을 명시함.",
            "constraints": ["1,000자 내외", "기술요소 및 평가계획 포함"],
            "output_format": "서술문 + 도식(기술흐름)"},
        {"section": "연차별 개발목표",
            "role": "PMO 리더",
            "query": "최종 목표 달성을 위한 연차별 및 기관별 개발 목표를 정량적으로 제시함. 각 연차별 KPI, 마일스톤, 검증 방법을 명확히 포함함.",
            "constraints": ["연차별 구분", "정량적 지표 포함"],
            "output_format": "표(연차/KPI/마일스톤/검증방법)"},
        {"section": "연차별 개발내용 및 범위",
        "role": "공동연구 컨소시엄 코디네이터",
        "query": "참여 기관별 역할, 책임, 연차별 산출물을 중복 및 누락 없이 기술함. 공동기관이 없는 경우 해당 항목은 생략함.",
        "constraints": ["기관별 역할 명확화", "중복/누락 금지"],
        "output_format": "표(기관/역할/산출물/책임)"},
        {"section": "추진방법 및 전략",
            "role": "총괄 아키텍트(Chief Architect)",
            "query": "핵심 기술개발 방법론, 예측 가능한 리스크와 대응 방안, 성능 검증 계획을 논리적으로 제시함. 기술의 우수성과 실현 가능성을 입증함.",
            "constraints": ["핵심 방법론 포함", "리스크 및 검증계획 명시"],
            "output_format": "서술문 + 표(리스크/대응방안)"},
        {"section": "과제 성과의 활용방안",
            "role": "사업개발 총괄(BD Head)",
            "query": "연구 성과가 실제 산업 및 시장에서 어떻게 활용될 수 있는지를 구체적으로 제시함. 목표 시장, 주요 수요처, 핵심 적용 시나리오, 기술의 차별화된 가치를 중심으로 사업화 방향을 설명함.",
            "constraints": ["시장 시나리오 포함", "Value Proposition 중심"],
            "output_format": "서술문 + 표(시장/수요처/적용시나리오)"},
        {"section": "신규사업 신설의 기대효과",
            "role": "거시경제 분석가(Macro-Economic Analyst)",
            "query": "본 과제가 국가 경제에 미치는 파급효과를 정량적 지표로 제시함. 시장 창출, 수입 대체, 수출 증대, 일자리 창출 등 거시적 효과를 수치로 증명함.",
            "constraints": ["정량적 수치 기반", "경제효과 명시"],
            "output_format": "표(지표/예상값/근거자료)"},
        {"section": "사회적 가치 창출 계획",
            "role": "사회적 가치 전략가(Social Value Strategist)",
            "query": "과제의 사회적 비전과 목표를 정의하고, 이를 달성하기 위한 구체적인 실행 로드맵을 수립함. 보건, 환경, 안전 등 사회적 가치 범주와 연계함.",
            "constraints": ["사회적 가치 범주 명시", "로드맵 포함"],
            "output_format": "서술문 + 표(목표/실행단계/성과지표)"},
        {"section": "사회적 가치창출의 기대효과",
            "role": "임팩트 평가 전문가(Impact Assessor)",
            "query": "사회적 가치 창출 계획이 실행되었을 때 예상되는 긍정적 변화를 정량적 및 정성적 임팩트 지표로 제시함. 사회적 파급효과를 객관적으로 설명함.",
            "constraints": ["정량/정성 지표 병기", "사회적 파급효과 포함"],
            "output_format": "표(지표유형/성과/측정방법)"},
        {"section": "경제적 성과창출의 기대효과",
            "role": "최고재무책임자(CFO)",
            "query": "기업 관점에서 본 과제의 재무적 성과를 구체적 수치와 함께 제시함. 예상 매출, 이익, 투자수익률(ROI), 순현재가치(NPV) 등 주요 지표를 근거와 함께 명료하게 기술함.",
            "constraints": ["재무지표 포함", "산출근거 명시"],
            "output_format": "표(지표/예상값/근거)"},
        {"section": "신규 인력 채용 계획 및 활용 방안",
            "role": "전략적 인사 파트너(Strategic HR Partner)",
            "query": "과제 수행에 필요한 핵심 인력의 채용, 배치, 교육 계획을 타임라인과 함께 제시함. 인력 확보 및 역량 극대화 방안을 구체적으로 기술함.",
            "constraints": ["타임라인 포함", "역량 강화 계획 명시"],
            "output_format": "표(직무/채용시점/교육계획)"},
        {"section": "보안등급의 분류 및 해당 사유",
            "role": "보안관리 책임자(Security Manager)",
            "query": "관련 법령 및 보안관리요령을 근거로 본 과제의 보안등급을 분류하고, 그 결정 사유를 간결하고 명확하게 기술함.",
            "constraints": ["법령 근거 포함", "사유 명시"],
            "output_format": "서술문"}
        ]

CHUNK_MAX = 500
CHUNK_OVERLAP = 50
TOPK = 5
REF_WEIGHTS = {
    "행정업무의 운영 및 혁신에 관한 규정(대통령령)": 0.30,
    "행정업무의 운영 및 혁신에 관한 규정 시행규칙": 0.30,
    "국가연구개발사업 연구개발계획서": 0.20,
    "전략계획서 작성안내서": 0.10,
    "Vertical": 0.10
}
DEFAULT_WEIGHT = 0.05

def guess_weight(filename: str) -> float:
    for k, w in REF_WEIGHTS.items():
        if k.lower() in filename.lower():
            return w
    return DEFAULT_WEIGHT

def chunk_text(txt: str, max_chars=CHUNK_MAX, overlap=CHUNK_OVERLAP):
    txt = " ".join(str(txt).split())
    chunks = []
    i = 0
    while i < len(txt):
        j = min(len(txt), i + max_chars)
        chunks.append(txt[i:j])
        if j == len(txt): break
        i = max(0, j - overlap)
    return chunks

def load_reference_chunks(law_dir: str):
    gc.collect()  # 파이썬 객체 메모리 수거
    with torch.cuda.device(1):
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    items = []
    for p in glob.glob(os.path.join(law_dir, "*")):
        text = ""
        try:
            ext = p.split(".")[-1].lower()
            if ext == "pdf":
                reader = PdfReader(p)
                for pg in reader.pages:
                    text += (pg.extract_text() or "") + "\n"
            elif ext in ("docx", "doc"):
                d = Document(p)
                text = "\n".join([x.text for x in d.paragraphs])
            elif ext in ("rtf", "txt"):
                with open(p, "r", encoding="utf-8", errors="ignore") as f:
                    text = f.read()
        except Exception:
            continue
        base = os.path.basename(p)
        weight = guess_weight(base)
        for idx, ck in enumerate(chunk_text(text)):
            items.append({"source": base, "chunk_id": idx, "weight": weight, "text": ck})
    return items



/home/alpaco/anaconda3/envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
embed_model = SentenceTransformer(E5_NAME, device=DEVICE)
REF_ITEMS = load_reference_chunks(LAW_DIR)
REF_EMBS = embed_model.encode(
    [it["text"] for it in REF_ITEMS],
    convert_to_tensor=True,
    normalize_embeddings=True
)
print(f"[INFO] Loaded {len(REF_ITEMS)} reference chunks.")

def search_reference(query: str, topk: int = TOPK):
    gc.collect()  # 파이썬 객체 메모리 수거
    with torch.cuda.device(1):
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    q = embed_model.encode(query, convert_to_tensor=True, normalize_embeddings=True)
    sims = util.cos_sim(q, REF_EMBS)[0]
    scores = [(float(s) * (1.0 + REF_ITEMS[i]["weight"]), i) for i, s in enumerate(sims)]
    scores.sort(key=lambda x: x[0], reverse=True)
    picks = []
    for sc, idx in scores[:topk]:
        it = REF_ITEMS[idx]
        picks.append({
            "source": it["source"], "chunk_id": it["chunk_id"],
            "weight": it["weight"], "score": round(sc, 4),
            "snippet": first_n_lines(it["text"], 400)
        })
    return picks


KeyboardInterrupt: 

In [5]:
del embed_model
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()


In [ ]:
# torch.cuda.set_device(1)  # 1번 GPU 선택
# gc.collect()
# torch.cuda.empty_cache()
# torch.cuda.ipc_collect()

In [7]:
import time
gen_tok = AutoTokenizer.from_pretrained(
    GEN_NAME,
    use_fast=False,           # fast tokenizer False유지 (Local호환성 문제)
    trust_remote_code=True
)
if gen_tok.pad_token_id is None:
    gen_tok.pad_token = gen_tok.eos_token


In [8]:
del gen_tok
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

time.sleep(2) 

In [ ]:
gen_model = AutoModelForCausalLM.from_pretrained(
    GEN_NAME,
    trust_remote_code=True,
    device_map=DEVICE
).eval()
gen_pipe = pipeline("text-generation", model=gen_model, tokenizer=gen_tok)

del gen_model
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

Loading checkpoint shards:   0%|          | 0/3 [00:02<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 260.00 MiB. GPU 1 has a total capacity of 23.69 GiB of which 12.94 MiB is free. Process 3042919 has 18.19 GiB memory in use. Including non-PyTorch memory, this process has 5.47 GiB memory in use. Of the allocated memory 4.97 GiB is allocated by PyTorch, and 213.81 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
del embed_model
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

NameError: name 'embed_model' is not defined